In [1]:
import numpy as np
from matplotlib import pyplot as plt
import time

In [21]:
# n_features = 16000
# n_samples = 15000
n_features = 3
n_samples = 10
n_classes = 2
X = np.random.randn(n_features, n_samples)
y = np.random.randint(0, n_classes, n_samples)
y = np.array([1 if i == 1 else -1 for i in y])
weights = np.random.randn(n_samples)


# Their method

In [29]:
s_t = time.time()

total_pos, total_neg = 0, 0
for w, label in zip(weights, y):
    if label == 1:
        total_pos += w
    else:
        total_neg += w

classifiers = []
total_features = X.shape[0]
for index, feature in enumerate(X):
    if len(classifiers) % 1000 == 0 and len(classifiers) != 0:
        print("Trained %d classifiers out of %d" % (len(classifiers), total_features))

    applied_feature = sorted(zip(weights, feature, y), key=lambda x: x[1])

    pos_seen, neg_seen = 0, 0
    pos_weights, neg_weights = 0, 0
    min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None
    current_idx = 0
    ws = []
    for w, f, label in applied_feature:
        ws.append(w)
        # min(all before current example are positive and all after are negative, all before current example are negative and all after are positive)
        # error = sum of weights of misclassified examples
        error = min(neg_weights + total_pos - pos_weights, pos_weights + total_neg - neg_weights)
        if error < min_error:
            min_error = error
            # best_feature = features[index]
            best_feature = (current_idx, f)
            best_threshold = f
            best_polarity = 1 if pos_seen > neg_seen else -1

        if label == 1:
            pos_seen += 1
            pos_weights += w
        else:
            neg_seen += 1
            neg_weights += w
        current_idx += 1

    print(np.array(ws))

    # clf = WeakClassifier(best_feature[0], best_feature[1], best_threshold, best_polarity)
    clf = best_feature, best_threshold, best_polarity, min_error
    classifiers.append(clf)

print("Time taken: %f seconds" % (time.time() - s_t))

[-0.39478424  0.50175924 -0.34804908  1.01640906 -0.98496301 -0.72845011
 -1.24018802  1.45116244 -0.19054711  0.29741805]
[ 0.50175924 -0.34804908 -0.72845011 -1.24018802  0.29741805  1.01640906
 -0.39478424 -0.19054711  1.45116244 -0.98496301]
[-1.24018802 -0.39478424  1.45116244 -0.98496301  0.29741805 -0.19054711
  0.50175924 -0.72845011  1.01640906 -0.34804908]
Time taken: 0.001689 seconds


# Mine

In [30]:
s_t = time.time()
classifiers2 = []
total_features2 = X.shape[0]
# TODO parallize this
for index, feature in enumerate(X):
    if len(classifiers2) % 1000 == 0 and len(classifiers2) != 0:
        print("Trained %d classifiers out of %d" % (len(classifiers2), total_features2))
        
    pos_weights, neg_weights = 0, 0
    min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None

    # sort by feature value first, then by weight, then by label
    sorting_indecies = np.lexsort((y, weights, feature))
    
    # s_w: srorted weights, s_f: sorted features, s_y: sorted labels
    s_w, s_f, s_y = weights[sorting_indecies], feature[sorting_indecies], y[sorting_indecies]
    # print(s_w)
    ## left = -1, right = 1
    # negative weights
    neg_weights = np.concatenate(([0], np.cumsum(s_w * (s_y == -1))))
    # positive weights (reversed)
    pos_weights = np.flip(np.cumsum(np.flip(s_w * (s_y == 1))))
    pos_weights2 = np.concatenate((pos_weights, [0]))
    
    idx1 = np.argmin(neg_weights + pos_weights2)
    if idx1 > len(neg_weights) - 1:
        idx1 = len(neg_weights) - 1
    
    min_error1 = neg_weights[idx1] + pos_weights2[idx1]

    best_feature1, best_threshold1 = (idx1, s_f[idx1]), s_f[idx1]

    ## left = 1, right = -1
    # repeat for s_y ==1, s_y == 1
    pos_weights_rl = np.concatenate(([0], np.cumsum(s_w * (s_y == 1))))

    neg_weights_rl = np.flip(np.cumsum(np.flip(s_w * (s_y == -1))))
    neg_weights_rl = np.concatenate((neg_weights_rl, [0]))
    idx2 = np.argmin(neg_weights_rl + pos_weights_rl)
    if idx2 > len(neg_weights_rl) - 1:
        idx2 = len(neg_weights_rl) - 1
    
    min_error2 = neg_weights_rl[idx2] + pos_weights_rl[idx2]
    print(neg_weights+ pos_weights2, 'idx1 = ', idx1, end=', ')
    print(neg_weights_rl+ pos_weights_rl, 'idx2 = ', idx2)

    best_feature2, best_threshold2 = (idx2, s_f[idx2]), s_f[idx2]
    

    if min_error1 < min_error2:
        min_error = min_error1
        best_feature = best_feature1
        best_threshold = best_threshold1
        best_polarity = -1
    else:
        min_error = min_error2
        best_feature = best_feature2
        best_threshold = best_threshold2
        best_polarity = 1

    clf2 = best_feature, best_threshold, best_polarity, min_error
    classifiers2.append(clf2)

    

print("Time taken: %f seconds" % (time.time() - s_t))

[ 0.47781287  0.08302862  0.58478787  0.93283694 -0.08357212 -1.06853513
 -1.79698524 -3.03717326 -1.58601082 -1.3954637  -1.09804565] idx1 =  7, [-1.09804565 -0.70326141 -1.20502065 -1.55306972 -0.53666066  0.44830235
  1.17675246  2.41694048  0.96577804  0.77523092  0.47781287] idx2 =  3
[ 0.47781287  0.97957211  1.32762118  0.59917107 -0.64101694 -0.34359889
 -1.36000795 -1.75479219 -1.56424508 -0.11308264 -1.09804565] idx1 =  7, [-1.09804565 -1.59980489 -1.94785397 -1.21940385  0.02078416 -0.27663389
  0.73977517  1.13455941  0.9440123  -0.50715014  0.47781287] idx2 =  2
[ 0.47781287 -0.76237515 -1.15715939  0.29400305 -0.69095996 -0.39354191
 -0.2029948   0.29876444 -0.42968567 -1.44609473 -1.09804565] idx1 =  9, [-1.09804565  0.14214237  0.53692661 -0.91423583  0.07072718 -0.22669087
 -0.41723799 -0.91899723 -0.19054711  0.82586194  0.47781287] idx2 =  0
Time taken: 0.003990 seconds


In [31]:
for el in classifiers:
    print(el)
print()
for el in classifiers2:
    print(el)

# print(classifiers[0])
# print(classifiers2[0])

((7, 0.4388840700609143), 0.4388840700609143, -1, -3.0371732568983996)
((2, 0.03704944486883128), 0.03704944486883128, -1, -1.947853965597011)
((9, 1.588808849994089), 1.588808849994089, -1, -1.4460947254201173)

((7, 0.4388840700609143), 0.4388840700609143, -1, -3.0371732568983996)
((2, 0.03704944486883128), 0.03704944486883128, 1, -1.947853965597011)
((9, 1.588808849994089), 1.588808849994089, -1, -1.4460947254201173)
